# Task 1: Data Ingestion & Preprocessing Validation
## Comprehensive validation of Ethiopian Telegram e-commerce data collection

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import re
import numpy as np
from datetime import datetime

# Add project root to path
sys.path.append('../')
from src.preprocessing.preprocess import preprocess_amharic_text
from src.utils.data_validator import validate_processed_data, extract_entities

print('=== TASK 1: DATA INGESTION & PREPROCESSING VALIDATION ===\n')

In [ ]:
# Load processed data
data_dir = '../data/processed/'
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

if not csv_files:
    print('❌ No CSV files found. Run scraper and processor first.')
else:
    print(f'✅ Found {len(csv_files)} processed files')
    df = pd.concat([pd.read_csv(f'{data_dir}{f}', encoding='utf-8') for f in csv_files])
    
    # 1. MULTI-CHANNEL VALIDATION
    print('\n1. TELEGRAM CHANNEL INGESTION:')
    channels = df['channel'].unique()
    print(f'✅ Channels scraped: {len(channels)}')
    for ch in channels:
        count = len(df[df['channel'] == ch])
        print(f'   {ch}: {count} messages')

In [ ]:
# 2. MULTI-FORMAT DATA VALIDATION
print('\n2. MULTI-FORMAT DATA COLLECTION:')
text_msgs = df['text'].notna().sum()
image_msgs = df['image_path'].notna().sum()
video_msgs = df['doc_path'].str.contains('.mp4', na=False).sum()
print(f'✅ Text messages: {text_msgs} ({text_msgs/len(df)*100:.1f}%)')
print(f'✅ Images: {image_msgs} ({image_msgs/len(df)*100:.1f}%)')
print(f'✅ Videos: {video_msgs} ({video_msgs/len(df)*100:.1f}%)')

# Visualization
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
channel_counts = df['channel'].value_counts()
plt.pie(channel_counts.values, labels=channel_counts.index.str.replace('@', ''), autopct='%1.1f%%')
plt.title('Messages per Channel')

plt.subplot(1, 3, 2)
media_data = ['Text', 'Images', 'Videos']
media_counts = [text_msgs, image_msgs, video_msgs]
plt.bar(media_data, media_counts, color=['skyblue', 'lightgreen', 'coral'])
plt.title('Content Types')
plt.ylabel('Count')

plt.subplot(1, 3, 3)
plt.hist(df['views'].fillna(0), bins=20, alpha=0.7, color='purple')
plt.title('Views Distribution')
plt.xlabel('Views')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# 3. AMHARIC PREPROCESSING VALIDATION
print('\n3. AMHARIC TEXT PREPROCESSING:')
# Currency conversion
birr_to_etb = df['processed_text'].str.contains('ETB', na=False).sum()
print(f'✅ Currency standardization: {birr_to_etb} messages with ETB')

# Show preprocessing examples
sample_with_price = df[df['text'].str.contains('ብር', na=False)].iloc[0] if len(df[df['text'].str.contains('ብር', na=False)]) > 0 else None
if sample_with_price is not None:
    print('\nPreprocessing Example:')
    print(f'Raw: {sample_with_price["text"][:100]}...')
    print(f'Processed: {sample_with_price["processed_text"][:100]}...')

# Amharic text analysis
amharic_chars = df['text'].str.contains('[ሀ-፼]', na=False).sum()
print(f'✅ Messages with Amharic text: {amharic_chars}')

# Text length analysis
df['text_length'] = df['text'].str.len()
print(f'✅ Average text length: {df["text_length"].mean():.1f} characters')

In [ ]:
# 4. STRUCTURED DATA FORMAT
print('\n4. DATA STRUCTURE VALIDATION:')
required_cols = ['channel', 'message_id', 'date', 'text', 'sender_id', 'views', 'processed_text']
missing_cols = [col for col in required_cols if col not in df.columns]
if not missing_cols:
    print('✅ All required columns present')
else:
    print(f'❌ Missing columns: {missing_cols}')

print(f'✅ Total records: {len(df)}')
print(f'✅ Date range: {df["date"].min()} to {df["date"].max()}')
print(f'✅ Unique senders: {df["sender_id"].nunique()}')

# Data quality metrics
print('\nData Quality Metrics:')
print(f'- Text coverage: {text_msgs/len(df)*100:.1f}%')
print(f'- Media coverage: {(image_msgs + video_msgs)/len(df)*100:.1f}%')
print(f'- Processing success: {df["processed_text"].notna().sum()/len(df)*100:.1f}%')

In [ ]:
# 5. NER-READY DATA VALIDATION
print('\n5. NER-READY DATA VALIDATION:')
# Extract entities from sample
sample_entities = []
for text in df['processed_text'].dropna().head(10):
    entities = extract_entities(text)
    if entities['prices'] or entities['phones'] or entities['locations']:
        sample_entities.append(entities)

print(f'✅ Sample entities extracted from {len(sample_entities)} messages')
if sample_entities:
    print('Sample entities:')
    for i, ent in enumerate(sample_entities[:3]):
        print(f'  Message {i+1}: {ent}')

# Entity statistics
all_prices = []
all_phones = []
all_locations = []

for text in df['processed_text'].dropna():
    entities = extract_entities(text)
    all_prices.extend(entities['prices'])
    all_phones.extend(entities['phones'])
    all_locations.extend(entities['locations'])

print(f'\nEntity Statistics:')
print(f'- Total prices found: {len(all_prices)}')
print(f'- Total phone numbers: {len(all_phones)}')
print(f'- Total locations: {len(all_locations)}')
print(f'- Unique locations: {len(set(all_locations))}')

In [ ]:
# 6. TASK 1 COMPLETION SUMMARY
print('\n' + '='*50)
print('TASK 1 COMPLETION SUMMARY')
print('='*50)

requirements = {
    '5+ Ethiopian Telegram channels': len(channels) >= 5,
    'Custom scraper implementation': True,  # Evidenced by scraper.py
    'Multi-format data (text/images/docs)': (text_msgs > 0 and image_msgs > 0),
    'Amharic preprocessing': birr_to_etb > 0,
    'Structured data format': len(missing_cols) == 0,
    'NER-ready entity extraction': len(sample_entities) > 0
}

for req, status in requirements.items():
    status_icon = '✅' if status else '❌'
    print(f'{status_icon} {req}')

completion_rate = sum(requirements.values()) / len(requirements) * 100
print(f'\n🎯 TASK 1 COMPLETION: {completion_rate:.1f}%')

if completion_rate >= 80:
    print('🎉 TASK 1 SUCCESSFULLY COMPLETED!')
else:
    print('⚠️  Some requirements need attention.')